In [ ]:
import os
os._exit(00)


In [2]:
!pip install transformers datasets torch

# Step 2: Import necessary libraries
import os
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

In [ ]:
import os

def load_ner_dataset(folder_path):
    sentences = []
    labels = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                tokens = []
                ner_tags = []
                for line in file:
                    if line.strip():
                        parts = line.strip().split()
                        token = " ".join(parts[:-1])
                        label = parts[-1]
                        tokens.append(token)
                        ner_tags.append(label)
                    else:
                        sentences.append(tokens)
                        labels.append(ner_tags)
                        tokens = []
                        ner_tags = []
                if tokens:
                    sentences.append(tokens)
                    labels.append(ner_tags)
    return sentences, labels


In [ ]:
folder_path = '/content/annotated'
sentences, labels = load_ner_dataset(folder_path)

In [ ]:
print(len(sentences ))
print(len(labels ))
print(sentences)
print(labels)

204
204
[['Medical', 'Record'], ['Patient', 'Information:'], ['Name:', 'John', 'Doe'], ['Date', 'of', 'Birth:', 'January', '15,', '1985'], ['Gender:', 'Male'], ['Patient', 'ID:', '123456'], ['Date', 'of', 'Visit:', 'March', '8,', '2000'], ['Chief', 'Complaint:'], ['Pain', 'in', 'the', 'neck', 'and', 'upper', 'back'], ['History', 'of', 'Present', 'Illness', '(HPI):'], ['Onset:', 'The', 'pain', 'started', 'approximately', '1', 'month', 'ago.'], ['Location:', 'Cervical', 'region', '(neck', 'and', 'upper', 'back).'], ['Duration:', 'Persistent,', 'worsening', 'over', 'time.'], ['Characteristics:', 'Dull,', 'aching', 'pain', 'with', 'occasional', 'sharp,', 'shooting', 'pains', 'down', 'the', 'arms.'], ['Aggravating', 'Factors:', 'Prolonged', 'sitting,', 'certain', 'neck', 'movements.'], ['Relieving', 'Factors:', 'Rest,', 'over-the-counter', 'pain', 'medications,', 'gentle', 'neck', 'exercises.'], ['Associated', 'Symptoms:', 'Numbness', 'and', 'tingling', 'in', 'the', 'arms,', 'occasional', '

In [ ]:
# Iterate over each sentence and its labels to collect unique labels
unique_labels = set(label for sublist in labels for label in sublist)

# Now create the tag-to-index mapping
tag2idx = {tag: idx for idx, tag in enumerate(unique_labels)}
tag2idx["PAD"] = len(tag2idx)  # Add "PAD" token for padding

print(tag2idx)

{'B-DIAGNOSE': 0, 'B-VISITDATE': 1, 'O': 2, 'I-DIAGNOSE': 3, 'I-VISITDATE': 4, 'PAD': 5}


In [ ]:
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize sentences and find the maximum length
tokenized_lengths = [len(tokenizer.encode(sentence, is_split_into_words=True)) for sentence in sentences]
max_length = max(tokenized_lengths)

print(f"Maximum length: {max_length}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Maximum length: 33


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Tokenize with padding to max_length
encoded_inputs = tokenizer(sentences, is_split_into_words=True, truncation=True, max_length=max_length, padding='max_length', return_tensors="pt")

print(encoded_inputs)


{'input_ids': tensor([[  101,  2657,  1647,  ...,     0,     0,     0],
        [  101,  5351,  1869,  ...,     0,     0,     0],
        [  101,  1271,   131,  ...,     0,     0,     0],
        ...,
        [  101, 10154,  1113,  ...,     0,     0,     0],
        [  101,  6182,  5351,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}


In [ ]:
# Check the tokenized output
print("Input IDs:", encoded_inputs['input_ids'])
print("Attention Mask:", encoded_inputs['attention_mask'])
print("Token Type IDs:", encoded_inputs.get('token_type_ids', 'Not provided'))
print(len(encoded_inputs['input_ids']))
print(len(encoded_inputs['attention_mask']))
print(len(encoded_inputs.get('token_type_ids', 'Not provided')))

Input IDs: tensor([[  101,  2657,  1647,  ...,     0,     0,     0],
        [  101,  5351,  1869,  ...,     0,     0,     0],
        [  101,  1271,   131,  ...,     0,     0,     0],
        ...,
        [  101, 10154,  1113,  ...,     0,     0,     0],
        [  101,  6182,  5351,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]])
Attention Mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])
Token Type IDs: tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
204
204
204


In [ ]:
# Decode input_ids to tokens
tokens = tokenizer.convert_ids_to_tokens(encoded_inputs['input_ids'][6])
print("Tokens:", tokens)


Tokens: ['[CLS]', 'date', 'of', 'visit', ':', 'march', '8', ',', '2000', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
from transformers import AutoModelForTokenClassification
import torch

def align_labels_with_tokens(labels, tokenized_inputs, label_all_tokens=True):
    aligned_labels = []
    for i, label in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens get ignored in the loss function
            elif word_idx != previous_word_idx:  # First token of the word
                label_ids.append(tag2idx[labels[i][word_idx]])
            else:  # Subsequent tokens of the word
                if label_all_tokens:
                    label_ids.append(tag2idx[labels[i][word_idx]])
                else:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        aligned_labels.append(label_ids)

    return aligned_labels

aligned_labels = align_labels_with_tokens(labels, encoded_inputs)
encoded_inputs['labels'] = torch.tensor(aligned_labels)

print(aligned_labels)
print(encoded_inputs['labels'])

[[-100, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 2, 2, 2, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(tag2idx)
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset

class NERDataset(Dataset):
    def __init__(self, encoded_inputs):
        self.input_ids = encoded_inputs['input_ids']
        self.attention_mask = encoded_inputs['attention_mask']
        self.labels = encoded_inputs['labels']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

dataset = NERDataset(encoded_inputs)


In [ ]:
def make_contiguous(module):
    for param in module.parameters():
        param.data = param.data.contiguous()

# Apply the function to the model after loading it
model.apply(make_contiguous)

from transformers import TrainerCallback

class ContiguousCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, **kwargs):
        # Make sure all model tensors are contiguous before each step
        model.apply(make_contiguous)



training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",   # Evaluates at the end of every epoch
    logging_strategy="steps",      # Log based on steps
    logging_steps=100,             # Log training loss every 100 steps
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3000,
    weight_decay=0.01,
    logging_dir='./logs',          # Log directory
    report_to="tensorboard",       # Reports to tensorboard (optional)
)

# Initialize Trainer with the ContiguousCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    callbacks=[ContiguousCallback()]
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.087811
2,No log,0.036063
3,No log,0.019884
4,No log,0.012627
5,No log,0.006072
6,No log,0.002990
7,No log,0.001608
8,0.046600,0.001088
9,0.046600,0.000751
10,0.046600,0.000593


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("./fine_tuned_ner_model")
tokenizer.save_pretrained("./fine_tuned_ner_model")


('./fine_tuned_ner_model/tokenizer_config.json',
 './fine_tuned_ner_model/special_tokens_map.json',
 './fine_tuned_ner_model/vocab.txt',
 './fine_tuned_ner_model/added_tokens.json',
 './fine_tuned_ner_model/tokenizer.json')

In [ ]:
# Ensure tensors are contiguous before saving
for param in model.parameters():
    param.data = param.data.contiguous()

# Save the model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

#Yeah Its Time for  Testing

In [ ]:
import nltk

# Download the punkt tokenizer models
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from nltk import sent_tokenize
import nltk

# Download the punkt tokenizer models (if not already downloaded)
nltk.download('punkt')

# Load the model and tokenizer
model_path = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

# Define your tag2idx dictionary (replace with your actual mapping)

idx2tag = {idx: tag for tag, idx in tag2idx.items()}

# Input text for NER
text = """Patient Information:
Medical Record
Name: John Doe
Date of Birth: January 15, 1985
Gender: Male
Patient ID: 123456
Date of Visit: Dec 5, 2019
Chief Complaint:
Pain in the right ankle and joint of the right foot
History of Present Illness (HPI):
Onset: The pain started approximately 3 weeks ago after a minor sprain.
Location: Right ankle and joint of the right foot.
Duration: Persistent, worsening with activity.
Characteristics: Sharp, throbbing pain with swelling and tenderness.
Aggravating Factors: Walking, standing for long periods, and physical activity.
Relieving Factors: Rest, ice application, elevation, and over-the-counter pain medications (ibuprofen). Associated Symptoms: Swelling, stiffness, and occasional bruising.
Past Medical History:
Chronic Conditions: Hypertension, Type 2 Diabetes.
Previous Surgeries: Appendectomy (2010).
Medications: Metformin, Lisinopril, Ibuprofen (as needed for pain).
Allergies: No known drug allergies.
Social History:
Occupation: Office worker.
Smoking: Non-smoker.
Alcohol Use: Occasionally.

Exercise: Infrequent.
Family History:
Father: Hypertension, Heart disease.
Mother: Type 2 Diabetes.
Siblings: Healthy.
Review of Systems:
General: No fever, no weight loss.
Musculoskeletal: Pain in the right ankle and joint of the right foot, no other joint pain or swelling.
Neurological: No numbness or tingling.
Cardiovascular: No chest pain, no palpitations.
Respiratory: No shortness of breath, no cough.
Physical Examination:
General: Alert, well-nourished, in no acute distress.
Vital Signs:
Blood Pressure: 130/85 mmHg.
Heart Rate: 78 bpm.
Respiratory Rate: 16 breaths/min.
Temperature: 98.6°F (37°C).
Inspection: Swelling and mild bruising around the right ankle and foot.
Palpation: Tenderness over the lateral aspect of the right ankle and dorsum of the right foot.
Range of Motion: Limited due to pain and swelling, especially in dorsiflexion and plantarflexion.
Strength: 4/5 in the right lower extremity.
Reflexes: Normal deep tendon reflexes in the lower extremities.
Sensation: Intact sensation in the right foot and ankle.
Special Tests: Negative anterior drawer test, negative talar tilt test indicating no major ligamentous instability.
Assessment:

Prognosis: Right ankle sprain with possible involvement of the joint of the right foot. Differential Prognosis: Ankle fracture, tendonitis, arthritis.
Plan:
Imaging: Order X-ray of the right ankle and foot to rule out fracture.
Medications: Continue ibuprofen for pain management; consider prescribing a stronger anti-inflammatory if necessary.
Referral: Refer to orthopedic specialist for further evaluation if no improvement.
Physical Therapy: Recommend physical therapy for rehabilitation and strengthening exercises.
Activity: Advise patient to avoid weight-bearing activities on the affected foot; use crutches if necessary.
Follow-Up: Schedule follow-up appointment in 2 weeks to review X-ray results and assess response to treatment.
Patient Education:
Discussed the nature of the condition, potential causes, and treatment options.
Instructed on proper use of medication and importance of adhering to prescribed therapy.
Encouraged patient to report any worsening symptoms or new concerns.

"""

# Split text into sentences to process each sentence separately
sentences = sent_tokenize(text)

# Prepare list to hold extracted entities
entities = []

# Iterate over sentences and perform NER
for sentence in sentences:
    # Tokenize the sentence with truncation
    inputs = tokenizer(sentence.split(), return_tensors="pt", is_split_into_words=True, truncation=True, max_length=512)

    # Get predictions from the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class indices
    predictions = torch.argmax(outputs.logits, dim=2)

    # Convert predicted indices to actual entity labels
    predicted_labels = [idx2tag[pred.item()] for pred in predictions[0]]

    # Get the tokens from the tokenizer
    tokens_chunk = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # # Debugging: Print tokens and their predicted labels to inspect
    print("\nSentence:", sentence)
    print("Tokens:", tokens_chunk)
    print("Predicted Labels:", predicted_labels)

    # Collect tokens and labels
    current_entity = []
    current_label = None

    for token, label in zip(tokens_chunk, predicted_labels):
        if token in ["[PAD]", "[CLS]", "[SEP]"]:
            continue

        # Handle 'B-' and 'I-' tokens (entity beginnings and continuations)
        if label.startswith("B-"):
            if current_entity:
                # Store the current entity before starting a new one
                entities.append((" ".join(current_entity), current_label))
            current_entity = [token]  # Start a new entity
            current_label = label[2:]  # Store the entity label without the 'B-'
        elif label.startswith("I-") and current_entity and label[2:] == current_label:
            current_entity.append(token)  # Continue the current entity
        else:
            if current_entity:
                entities.append((" ".join(current_entity), current_label))
            current_entity = []  # Clear the current entity
            current_label = None

    # Append the last entity
    if current_entity:
        entities.append((" ".join(current_entity), current_label))

# Display the extracted entities
if entities:
    for entity, label in entities:
        print(f"Entity: {entity}, Label: {label}")
else:
    print("No entities were extracted.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Sentence: Patient Information:
Medical Record
Name: John Doe
Date of Birth: January 15, 1985
Gender: Male
Patient ID: 123456
Date of Visit: Dec 5, 2019
Chief Complaint:
Pain in the right ankle and joint of the right foot
History of Present Illness (HPI):
Onset: The pain started approximately 3 weeks ago after a minor sprain.
Tokens: ['[CLS]', 'patient', 'information', ':', 'medical', 'record', 'name', ':', 'j', '##oh', '##n', 'do', '##e', 'date', 'of', 'birth', ':', 'j', '##anu', '##ary', '15', ',', '1985', 'gender', ':', 'male', 'patient', 'id', ':', '123', '##45', '##6', 'date', 'of', 'visit', ':', 'de', '##c', '5', ',', '2019', 'chief', 'complaint', ':', 'pain', 'in', 'the', 'right', 'ankle', 'and', 'joint', 'of', 'the', 'right', 'foot', 'history', 'of', 'present', 'illness', '(', 'hp', '##i', ')', ':', 'onset', ':', 'the', 'pain', 'started', 'approximately', '3', 'weeks', 'ago', 'after', 'a', 'minor', 's', '##pra', '##in', '.', '[SEP]']
Predicted Labels: ['O', 'O', 'O', 'O', 'O', 